In [ ]:

from matplotlib import pyplot as plt

from src.image.video.snapshot import VideoCamera
from src.image.object_detection.keras_detector import KerasDetector



# Image Demo

WELCOME MESSAGE

## Recognizing objects in images/video

In the cell below you can run a program that uses AI to recognize objects in images. Try to see if you can make it recognize some objects around you and perhaps find some objects that it can not detect. 


`Program for object recognition.`

In [ ]:
# In this cell, the user should be able to write the labels from the object recognitions program to a file

### Creating an experiment

1. Using Python in the cell above, make sure the labels from the program are stored to a file.
1. Select a number of object for testing the object recognition program.
1. Run the program while showing it the object you selected. Make sure to show all the objects and to time (approximately) when you showed each of the objects.

### Analysing the experiment

1. In the cell below, reload the file with the experiment data

`Make some kind of usefull visualization, which they can easily access.`

In [ ]:
plt.close("all")
plt.ion()
the_video = VideoWText(
    record_frames=True, 
    frame_rate=15, 
    seconds=10,
)

## 1 Object classification


### Taking pictures

Run the following cell and take multiple pictures of five objects of your choice after labeling the object

In [ ]:
%matplotlib inline
%matplotlib auto
num_pictures_each = 2
num_objects = 1
num_labels_predicted = 4

frames = []
labels = []
for i in range(num_objects):
    label_name = input("Object label: ")
    labels.append(label_name)

    my_camera = VideoCamera(n_photos = num_pictures_each,title = "Hold object before camera to take picture and press enter", block = True)

    my_camera.start()
    frames.append(my_camera.photos[:num_pictures_each])


### Classifying images


We load a neural network to categorize your object pictures

In [ ]:
net = KerasDetector(model_name="inception_resnet_v2",n_labels_returned = num_labels_predicted)

In [ ]:
%matplotlib inline
plt.close("all")

f, axarr = plt.subplots(num_objects, num_pictures_each, figsize=(4*num_pictures_each,3*num_objects))
for i in range (num_objects):
    print(100*"_" +"\nObject {}".format(labels[i]) )
    for j in range(num_pictures_each):
        if j==0:
            axarr[i,j].set_ylabel(labels[i], rotation=0, size='large', labelpad=5*(len(labels[i])))
        axarr[i,j].set_title("Picture {}".format(j))
        probable_labels=net.label_frame(frames[i][j])[0]
        print("Picture {}: ".format(j),", ".join(probable_labels))
        axarr[i,j].imshow(frames[i][j])
        axarr[i,j].tick_params( which='both', labelbottom=False,labelleft = False,length=0)


### Analyzing the results
1. How accurate are the labels for each object? Decide if one of the probable labels accurately describes the object and  count how many images are correct for each object. 

1. Are there differences in the accuracies for the objects? 

1. If yes, why do you think this happens?

### Checking against categories

1. Find a list of the categories used for training the neural network

1. Do the categories reflect the environment you are in? 

1. Are there categories that you would not expect to encounter? 

1. Can you find an object  that you can not find in the categories? 

1. What do you expect to happen when you classify this object? Take pictures of the object and classify them? What does happen?